## Задание 1

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [10]:
import pandas as pd

LOW_RATING_NAME = 'low'
AVERAGE_RATING_NAME = 'average'
HIGTH_RATING_NAME = 'hight'

def set_rating(rating: float) -> str:
    if rating <= 2:
        return LOW_RATING_NAME
    
    elif rating <= 4:
        return AVERAGE_RATING_NAME
    
    return HIGTH_RATING_NAME


df = pd.read_csv('./movies/ratings.csv')
df['class'] = df.rating.apply(set_rating)
df.head()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,average
1,1,1029,3.0,1260759179,average
2,1,1061,3.0,1260759182,average
3,1,1129,2.0,1260759185,low
4,1,1172,4.0,1260759205,average


## Задание 2

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [31]:
import pandas as pd

UNDEFINED_REGION_NAME = 'undefined'

GEO_DATA = {
    'Центр': [
        'москва',
        'тула',
        'ярославль'
    ],
    'Северо-Запад': [
        'петербург',
        'псков',
        'мурманск'
    ],
    'Дальний Восток': [
        'владивосток',
        'сахалин',
        'хабаровск'
    ] 
}

def identify_region(query: str, regions_map: dict) -> str:
    for region_name, cityes_list in regions_map.items():
        if any(city in query for city in cityes_list):
            return region_name

    return UNDEFINED_REGION_NAME

df = pd.read_csv('keywords.csv')

df['region'] = df.keyword.apply(identify_region, regions_map=GEO_DATA)

df_to_show = df.sort_values(by='region', ascending=False)

In [33]:
df_to_show.head()

,keyword,shows,region
6999,москва 24,37459,Центр
61986,такси везет москва официальный сайт,6063,Центр
43404,суперджоб москва вакансии,8208,Центр
73226,фанатик спартак москва,5353,Центр
68005,гостиница москва,5651,Центр


In [38]:
df_to_show.tail()

,keyword,shows,region
33455,тобот мультфильм на русском смотреть онлайн,9790,undefined
33454,логические загадки,10369,undefined
33453,пенсионный фонд рф,9875,undefined
33452,olesya fattakhova,9832,undefined
99999,ггму,3630,undefined


## Задание 3

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [179]:
import re
import pandas as pd

FOUR_DIGIT_REG_EXP = '\d{4}'
DEFAULT_YEAR = 1900
TASK_MIN_YEAR = 1950
TASK_MAX_YEAR = 2010

def get_year(title: str, min_lim: int, max_lim: int) -> int:
    match = re.search(FOUR_DIGIT_REG_EXP, title)
    current_year = int(match[0] if match else 0)
    
    if min_lim <= current_year <= max_lim:
        return current_year
    
    return DEFAULT_YEAR

ratings_df = pd.read_csv('./movies/ratings.csv')
movies_df = pd.read_csv('./movies/movies.csv')

movies_df['years'] = movies_df.title.apply(
    get_year,
    min_lim=TASK_MIN_YEAR,
    max_lim=TASK_MAX_YEAR
)

# filtering df for optimisation
movies_df.query(
    f'{TASK_MIN_YEAR} <= years <= {TASK_MAX_YEAR}',
    inplace=True
)
    
movie_rating_years_df = (
    ratings_df
    .merge(movies_df, on='movieId')
    [[ 'title', 'rating', 'years']]
)

(
    movie_rating_years_df
    .groupby('years')
    ['rating']
    .agg(['count', 'mean'])
    ['mean']
    .sort_values(ascending=False)
)

years
1957    4.014241
1972    4.011136
1952    4.000000
1974    3.999058
1954    3.994220
1951    3.983539
1962    3.952446
1950    3.915254
1977    3.905786
1964    3.841492
1959    3.841033
1958    3.823171
1969    3.817376
1967    3.814516
1975    3.814147
1960    3.809211
1976    3.806794
1973    3.806667
1979    3.783201
1981    3.758763
1953    3.754587
1971    3.753532
1955    3.751724
1966    3.736726
1980    3.708816
1963    3.705966
1970    3.700000
1968    3.685714
1965    3.678715
1961    3.663851
          ...   
1984    3.657390
1982    3.639225
1991    3.628265
1956    3.612426
1987    3.606687
1985    3.586653
1988    3.552990
1978    3.549003
2007    3.548437
1994    3.537963
1993    3.536664
2010    3.534717
1989    3.534349
2009    3.534267
1986    3.528265
2006    3.511478
2008    3.498057
2004    3.491649
2001    3.476130
1998    3.474388
1999    3.473070
1995    3.472118
1992    3.466754
2002    3.456297
1990    3.448718
2005    3.448434
2003    3.445843
1996    